In [1]:
import numpy as np
import pandas as pd
import musicbrainzngs
import matplotlib.pyplot as plt
import re
from fuzzywuzzy import fuzz
from tqdm.notebook import tqdm, trange
import time
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [2]:
def getHTMLContent(link):
    html = urlopen(link)
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [3]:
def getTable(id):
    content = getHTMLContent(f'https://bestsellingalbums.org/artist/{id}')
    artist = content.find("title").contents[0].replace(" album sales", "")
    table = content.find_all('table')[0]
    rows = table.find_all('tr')
    #df = pd.DataFrame(columns=["Sales","Albums","Country"])
    df = pd.DataFrame(columns=["Artist","Sales","Country"])
    for i,row in enumerate(rows[1:]):
        df.loc[i] = [artist,
                     row.find(id="t_sales").contents[0],
                     #row.find(id="t_albums_count").contents[0],
                     row.find("div", {"class": "mobile_sales_country"}).contents[1]]
    return df

Set the musicbrainz user agent for use with API

In [37]:
musicbrainzngs.set_useragent("Python-urllib", "3.8")

## Get the data
Scrape the bestsellingalbums.org site for a list of artists and sales data

In [ ]:
# takes 2 seconds per artist, so split into 3 (~4 hours)

max_id = 14674

ix = 0 # 0, 1, 2

start = ix*5000
end = np.min([start + 5000, max_id])

df_full = pd.DataFrame()

for i in trange(start, end): # 14674
    try:
        df = getTable(i+1)
    except:
        pass
    else:
        df_full = pd.concat([df_full, df])
    time.sleep(.1)
    
df_full.to_csv(f'data/album_sales_{ix}.csv', index=False)

Compile the data

In [66]:
df_0 = pd.read_csv('data/album_sales_0.csv')
df_1 = pd.read_csv('data/album_sales_1.csv')
df_2 = pd.read_csv('data/album_sales_2.csv')
df = pd.concat([df_0, df_1, df_2], ignore_index=True)
df['Country'] = df['Country'].str.strip()
df['Sales'] = df['Sales'].str.replace(',', '')
df['Sales'] = df['Sales'].astype('int')
df.shape

(29203, 3)

In [67]:
df.head()

,Artist,Sales,Country
0,19,2600000,Japan
1,23,10000,Austria
2,23,100000,Germany
3,112,35000,Australia
4,112,150000,Canada


### Scale sales by size of market

In [68]:
sales_by_country = df.groupby('Country')['Sales'].sum()
avg_total_sales = sales_by_country.mean()
#sales_by_artist = df.groupby('Artist')['Sales'].sum()
#pd.DataFrame(sales_by_country).to_csv('sales_by_country.csv')
df['country_sales'] = list(sales_by_country[df['Country']])
df['Scaled'] = round(df['Sales'] / df['country_sales'] * avg_total_sales, -4)

In [69]:
df['Sales'].sum(), df['Scaled'].sum()

(22074970548, 22074430000.0)

In [70]:
# df['country_sales'] = list(sales_by_country[df['Country']])
# df['artist_sales'] = list(sales_by_artist[df['Artist']])
# df['scaled_a'] = df['Sales'] / (df['country_sales'] / df['Sales'].sum())
# scaled_by_artist = df.groupby('Artist')['scaled_a'].sum()
# df['scaled_a_total'] = list(scaled_by_artist[df['Artist']])
# df['Scaled'] = round(df['scaled_a'] * (df['artist_sales'] / df['scaled_a_total']), 0)

In [71]:
#pd.DataFrame(sales_by_country).to_csv('sales_by_country.csv')

### Cleanup countries

In [72]:
df_pop = pd.read_html('https://www.worldometers.info/world-population/population-by-country/')[0]

In [73]:
df_pop.head()

,#,Country (or dependency),Population (2020),Yearly Change,Net Change,Density (P/Km²),Land Area (Km²),Migrants (net),Fert. Rate,Med. Age,Urban Pop %,World Share
0,1,China,1439323776,0.39 %,5540090,153,9388211,-348399.0,1.7,38,61 %,18.47 %
1,2,India,1380004385,0.99 %,13586631,464,2973190,-532687.0,2.2,28,35 %,17.70 %
2,3,United States,331002651,0.59 %,1937734,36,9147420,954806.0,1.8,38,83 %,4.25 %
3,4,Indonesia,273523615,1.07 %,2898047,151,1811570,-98955.0,2.3,30,56 %,3.51 %
4,5,Pakistan,220892340,2.00 %,4327022,287,770880,-233379.0,3.6,23,35 %,2.83 %


In [74]:
countries_pop = pd.unique(df_pop['Country (or dependency)'])
countries_df = pd.unique(df['Country'])
#countries_df.to_csv('population_by_country_2020.csv')

In [75]:
unmatched = [c for c in countries_df if not c in countries_pop]

In [76]:
unmatched

['Korea',
 'VK Music',
 'Grecce',
 'Yugoslavia',
 'Chilie',
 'Czech Republic',
 'USSR',
 'Gulf States',
 'Czechoslovakia',
 'VK global playlists',
 'Spotify',
 'Ex Yugoslavia',
 'VK Singles']

In [77]:
drop_countries = ['VK Music','Yugoslavia','Gulf States','VK global playlists','Ex Yugoslavia','Spotify', 'VK Singles']

In [78]:
df = df[~df['Country'].isin(drop_countries)]

In [79]:
df['Country'] = df['Country'].replace({
    'Korea': 'South Korea',
    'Grecce': 'Greece',
    'Chilie': 'Chile',
    'USSR': 'Russia',
    'Czechoslovakia': 'Czech Republic'}
)

df_pop['Country (or dependency)'] = df_pop['Country (or dependency)'].replace({
    'Czech Republic (Czechia)': 'Czech Republic'}
)

In [80]:
df_pop = df_pop.rename({'Country (or dependency)': 'Country', 'Population (2020)': 'Population'}, axis=1)
df_pop[['Country', 'Population']].to_csv('data/population_by_country_2020.csv', index=False)

### Convert to wide format
1 row per artist, countries as columns

In [271]:
df_wide = df.pivot_table(index="Artist", columns="Country", values=["Sales", "Scaled"])
df_wide = df_wide.fillna(0)
#df_wide.columns = df_wide.columns.str.strip()

In [272]:
df_wide['Total'] = df_wide['Sales'].sum(axis=1)
df_wide['Total Scaled'] = df_wide['Scaled'].sum(axis=1)

In [273]:
#df_wide.columns = df_wide.columns.map('|'.join).str.strip('|')

In [274]:
df_wide.head()

Sales                                             \
Country               Argentina Australia Austria  Belgium Brazil Bulgaria   
Artist                                                                       
#LIKEME CAST                0.0       0.0     0.0  50000.0    0.0      0.0   
$-CREW                      0.0       0.0     0.0      0.0    0.0      0.0   
& THE CITY SERIES           0.0       0.0     0.0      0.0    0.0      0.0   
(G)I-DLE ((여자)아이들)          0.0       0.0     0.0      0.0    0.0      0.0   
-M- (MATTHIEU CHEDID)       0.0       0.0     0.0  25000.0    0.0      0.0   

                                                   ... Scaled                  \
Country               Canada Chile China Colombia  ... Taiwan Thailand Turkey   
Artist                                             ...                          
#LIKEME CAST             0.0   0.0   0.0      0.0  ...    0.0      0.0    0.0   
$-CREW                   0.0   0.0   0.0      0.0  ...    0.0      0.0    0.0   
& THE CITY SERIES        0.0   0.0   0.0      0.0  ...    0.0      0.0    0.0   
(G)I-DLE ((여자)아이들)       0.0   0.0   0.0      0.0  ...    0.0      0.0    0.0   
-M- (MATTHIEU CHEDID)    0.0   0.0   0.0      0.0  ...    0.0      0.0    0.0   

                                                                              \
Country               Ukraine United Kingdom United States Uruguay Venezuela   
Artist                                                                         
#LIKEME CAST              0.0            0.0           0.0     0.0       0.0   
$-CREW                    0.0            0.0           0.0     0.0       0.0   
& THE CITY SERIES         0.0            0.0           0.0     0.0       0.0   
(G)I-DLE ((여자)아이들)        0.0            0.0           0.0     0.0       0.0   
-M- (MATTHIEU CHEDID)     0.0            0.0           0.0     0.0       0.0   

                           Total Total Scaled  
Country                                        
Artist                                         
#LIKEME CAST             50000.0     290000.0  
$-CREW                  100000.0      40000.0  
& THE CITY SERIES        68833.0     240000.0  
(G)I-DLE ((여자)아이들)      507190.0     400000.0  
-M- (MATTHIEU CHEDID)  2275000.0    1030000.0  

[5 rows x 112 columns]

In [275]:
df_wide.shape

(14232, 112)

### Filter by total EN sales
Filter to only include artists with significant sales in English speaking countries

In [276]:
en_countries = ['United Kingdom', 'United States', 'Canada', 'Australia', 'New Zealand', 'Ireland']
en_sales = ['Sales|' + c for c in en_countries]
min_en_sales = 500000 

In [277]:
df_wide.columns = df_wide.columns.map('|'.join).str.strip('|')

In [278]:
df_wide = df_wide[df_wide[en_sales].sum(axis=1) >= min_en_sales]

In [279]:
df_wide.shape

(3082, 112)

In [280]:
df_wide.head()

,Sales|Argentina,Sales|Australia,Sales|Austria,Sales|Belgium,Sales|Brazil,Sales|Bulgaria,Sales|Canada,Sales|Chile,Sales|China,Sales|Colombia,...,Scaled|Taiwan,Scaled|Thailand,Scaled|Turkey,Scaled|Ukraine,Scaled|United Kingdom,Scaled|United States,Scaled|Uruguay,Scaled|Venezuela,Total,Total Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
10 YEARS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10000.0,0.0,0.0,500000.0,10000.0
"10,000 MANIACS",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30000.0,220000.0,0.0,0.0,8180000.0,250000.0
100 HITS SERIES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,610000.0,0.0,0.0,0.0,3840000.0,610000.0
100% HITS SERIES (AUSTRALIA),0.0,3220000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3220000.0,3000000.0
100% SERIES (UK),0.0,0.0,0.0,0.0,0.0,0.0,100000.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,350000.0,0.0,0.0,0.0,2340000.0,390000.0


## Get artist info from musicbrainz

In [281]:
def get_genres(tags, genre_list, max_genres=3):
    df_tag = pd.DataFrame(tags)
    df_tag['count'] = df_tag['count'].astype(int)
    df_tag['name'] = df_tag['name'].str.replace('[,.;@#?!$-]', ' ', regex=True)
    df_tag['name'] = df_tag['name'].str.replace('new wave', 'new-wave', regex=True)
    df_tag['name'] = df_tag['name'].str.replace('hip hop', 'hip-hop', regex=True)
    df_tag = df_tag.sort_values('count', ascending=False)
    df_tag = df_tag[df_tag['count']>0].head(max_genres)
    
    my_tags = []
    for tag in df_tag['name']:
        tag_words = tag.split(' ')
        for word in tag_words:
            if word in genre_list:
                my_tags.append(word)
    return ';'.join(set(my_tags))

In [282]:
# result = musicbrainzngs.search_artists(artist='a flock of seagulls')
# get_genres(result['artist-list'][0]['tag-list'], approved_genres, 3)

In [283]:
approved_genres = ['pop','rock','punk','indie','alternative','metal','jazz','blues','electronic','hip-hop','country',
                   'soul','classical','latin','folk','r&b','reggae','ska','dance','rap','new-wave'
                  ]

for artist in tqdm(df_wide.index):
    result = musicbrainzngs.search_artists(artist=artist)

    artist_id = None
    artist_name = None
    country_name = None
    area_name = None
    genre_name = None
    tag_count = 0
    
    try:
        artist_id = result['artist-list'][0]['id']
    except:
        pass
    try:
        artist_name = result['artist-list'][0]['name']
    except:
        pass
    try:
        country_name = result['artist-list'][0]['country']
    except:
        pass
    try:
        area_name = result['artist-list'][0]['area']['name']
    except:
        pass
    try:
        genre_name = get_genres(result['artist-list'][0]['tag-list'], approved_genres)

    except:
        pass
    
    df_wide.loc[artist, 'ID'] = artist_id
    df_wide.loc[artist, 'Artist_name'] = artist_name
    df_wide.loc[artist, 'Country'] = country_name
    df_wide.loc[artist, 'Area'] = area_name
    df_wide.loc[artist, 'Genre'] = genre_name
    
    time.sleep(.1)

  0%|          | 0/3082 [00:00<?, ?it/s]

In [284]:
df_wide.head()

,Sales|Argentina,Sales|Australia,Sales|Austria,Sales|Belgium,Sales|Brazil,Sales|Bulgaria,Sales|Canada,Sales|Chile,Sales|China,Sales|Colombia,...,Scaled|United States,Scaled|Uruguay,Scaled|Venezuela,Total,Total Scaled,ID,Artist_name,Country,Area,Genre
Artist,,,,,,,,,,,,,,,,,,,,,
10 YEARS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10000.0,0.0,0.0,500000.0,10000.0,b18bc9c4-6f22-4f1b-a918-e9c86a39fe7a,10 Years,US,United States,metal;rock;alternative
"10,000 MANIACS",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,220000.0,0.0,0.0,8180000.0,250000.0,b9a06530-1241-4162-836f-7b8e79deaa58,"10,000 Maniacs",US,United States,indie;rock;alternative;pop
100 HITS SERIES,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3840000.0,610000.0,40ab4b9c-7740-4d22-85bf-d48ce0c3db59,Series,None,Melbourne,None
100% HITS SERIES (AUSTRALIA),0.0,3220000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3220000.0,3000000.0,8a63d651-0fa9-4b94-8569-e180899c427a,AustraliA,None,None,None
100% SERIES (UK),0.0,0.0,0.0,0.0,0.0,0.0,100000.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2340000.0,390000.0,a57d7a1b-b7ef-4f36-bbf6-9f5a167cd08d,UK Subs,GB,United Kingdom,rock;punk


In [285]:
df_wide.to_csv('data/album_sales_wide_0.csv')

### Cleanup artist
Cleanup artists who don't match the musicbrainz artist

In [2]:
df_wide = pd.read_csv('data/album_sales_wide_0.csv')

In [3]:
df_wide['Artist_name'] = df_wide['Artist_name'].str.upper()
df_wide['Artist_name'] = df_wide['Artist_name'].str.replace('&','AND')
df_wide['Artist_name'] = df_wide['Artist_name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_wide['Artist_name'] = df_wide['Artist_name'].str.replace('[^\w\s]',' ', regex=True)
df_wide['Artist OG'] = df_wide['Artist']
df_wide['Artist'] = df_wide['Artist'].str.replace('&','AND')
df_wide['Artist'] = df_wide['Artist'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_wide['Artist'] = df_wide['Artist'].str.replace('[^\w\s]',' ', regex=True)

In [4]:
df_wide['artist_match'] = df_wide.apply(lambda x: fuzz.ratio(x['Artist'], x['Artist_name']), axis=1)

In [5]:
df_wide.to_csv('data/album_sales_wide_1.csv', index=False)

Time to do some manual cleanup

In [6]:
df_wide = pd.read_csv('data/album_sales_wide_1.csv', index_col='Artist')
df_remove = pd.read_csv('data/artist_remove.csv', index_col='Artist')
df_update = pd.read_csv('data/artist_update.csv', index_col='Artist')

In [7]:
df_remove.head()

,ID,Artist_name,Country,Area,Genre,Artist OG,artist_match
Artist,,,,,,,
100 HITS SERIES,40ab4b9c-7740-4d22-85bf-d48ce0c3db59,SERIES,NaN,Melbourne,other,100 HITS SERIES,57
100 HITS SERIES AUSTRALIA,8a63d651-0fa9-4b94-8569-e180899c427a,AUSTRALIA,NaN,NaN,other,100% HITS SERIES (AUSTRALIA),49
100 SERIES UK,a57d7a1b-b7ef-4f36-bbf6-9f5a167cd08d,UK SUBS,GB,United Kingdom,punk,100% SERIES (UK),26
101 HITS SERIES UK,5c176092-cb4d-4e05-806b-1e9414f2b28c,101 STRINGS,US,United States,other,101 HITS SERIES (UK),52
4 HIM,4cc51547-88ae-49a2-bf7e-0ef65ee7a94c,HIM,FI,Finland,pop rock,4 HIM,75


In [8]:
df_update.head()

,ID,Artist_name,Country,Area,Genre,Artist OG,artist_match
Artist,,,,,,,
B O B,94c338ff-1985-4429-9dc8-997b61bb5932,B O B,US,United States,hip hop,B.O.B.,53
BELLY,c118bc97-11a7-41dc-a55e-48c3bcf22ac2,BELLY,US,United States,rock,BELLY,67
BIG PUNISHER,609e7afd-3552-4102-9501-7611858ea320,BIG PUN,US,United States,hip hop,BIG PUNISHER,80
BOW WOW,6fd44ae7-097d-4979-94ba-5dfc40d9f7ad,BOW WOW,US,United States,hip hop,BOW WOW,78
BUDDY HOLLY AND THE CRICKETS,d352f5dd-3023-4565-a7bb-52396bf8821d,BUDDY HOLLY,US,United States,pop;rock,BUDDY HOLLY & THE CRICKETS,60


In [9]:
df_wide = df_wide[~df_wide.index.isin(df_remove.index)]

In [10]:
df_wide.update(df_update)

In [11]:
df_wide.to_csv('data/album_sales_wide_2.csv')

In [12]:
#df_wide.columns

In [13]:
#df_wide = df_wide.drop(['Spotify','VK Music','VK Singles', 'VK global playlists'], axis=1)

### Cleanup origin country

In [14]:
df_wide = pd.read_csv('data/album_sales_wide_2.csv', index_col='Artist')

In [15]:
df_wide.shape

(2939, 119)

In [16]:
df_wide.head()

,Sales|Argentina,Sales|Australia,Sales|Austria,Sales|Belgium,Sales|Brazil,Sales|Bulgaria,Sales|Canada,Sales|Chile,Sales|China,Sales|Colombia,...,Scaled|Venezuela,Total,Total Scaled,ID,Artist_name,Country,Area,Genre,Artist OG,artist_match
Artist,,,,,,,,,,,,,,,,,,,,,
10 YEARS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,500000.0,10000.0,b18bc9c4-6f22-4f1b-a918-e9c86a39fe7a,10 YEARS,US,United States,metal;rock;alternative,10 YEARS,100.0
10 000 MANIACS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,8180000.0,250000.0,b9a06530-1241-4162-836f-7b8e79deaa58,10 000 MANIACS,US,United States,indie;rock;alternative;pop,"10,000 MANIACS",100.0
10CC,0.0,0.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,...,0.0,1965300.0,860000.0,f37c537b-3557-4031-bfd6-ab63ced32854,10CC,GB,United Kingdom,rock;pop,10CC,100.0
112,0.0,35000.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,...,0.0,5745000.0,250000.0,9132d515-dc0e-4494-85ae-20f06eed14f9,112,US,United States,pop;r&b,112,100.0
2 CHAINZ,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,0.0,0.0,0.0,...,0.0,2040000.0,80000.0,dff0d392-4cd5-4052-9fbb-f485df3891e5,2 CHAINZ,US,United States,hip-hop,2 CHAINZ,100.0


In [17]:
# Source: https://simplemaps.com/data/world-cities
df_wide['Country'].isna().sum()

164

In [18]:
city_lookup = pd.read_csv('data/worldcities.csv')
city_lookup = city_lookup.drop_duplicates('city')
city_lookup.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629


In [19]:
df_wide = df_wide.reset_index().merge(city_lookup[['city','iso2']], how='left', left_on='Area', right_on='city')
df_wide.loc[df_wide['Country'].isna(), "Country"] = df_wide.loc[df_wide['Country'].isna(), "iso2"]

In [20]:
df_wide.shape

(2939, 122)

In [21]:
df_wide['Country'].isna().sum()

95

In [22]:
df_wide.loc[df_wide['Country'].isna(), "Area"].value_counts()[:10]

England             14
Québec               9
Scotland             5
Northern Ireland     3
Saddle River         2
Hawaii               2
Wrexham              1
Monte-Carlo          1
Mazatlan             1
Alabama              1
Name: Area, dtype: int64

In [23]:
df_wide['Country'] = df_wide['Country'].fillna(df_wide['Area'].replace({
    'England': 'GB',
    'Québec': 'CA',
    'Scotland': 'GB',
    'Wales': 'GB',
    'Northern Ireland': 'GB',
    'Saddle River': 'US',
    'Hawaii': 'US'}
))

In [24]:
df_countries = city_lookup[['iso2','country']].drop_duplicates()

In [25]:
df_wide['Country Code'] = df_wide['Country']
df_wide['Country'] = df_wide.merge(df_countries, how='left', left_on='Country', right_on='iso2')['country']

In [26]:
df_wide = df_wide[df_wide['Country Code'].isin(df_countries['iso2'])]

In [27]:
df_wide.head()

,Artist,Sales|Argentina,Sales|Australia,Sales|Austria,Sales|Belgium,Sales|Brazil,Sales|Bulgaria,Sales|Canada,Sales|Chile,Sales|China,...,ID,Artist_name,Country,Area,Genre,Artist OG,artist_match,city,iso2,Country Code
0,10 YEARS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,b18bc9c4-6f22-4f1b-a918-e9c86a39fe7a,10 YEARS,United States,United States,metal;rock;alternative,10 YEARS,100.0,NaN,NaN,US
1,10 000 MANIACS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,b9a06530-1241-4162-836f-7b8e79deaa58,10 000 MANIACS,United States,United States,indie;rock;alternative;pop,"10,000 MANIACS",100.0,NaN,NaN,US
2,10CC,0.0,0.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,...,f37c537b-3557-4031-bfd6-ab63ced32854,10CC,United Kingdom,United Kingdom,rock;pop,10CC,100.0,NaN,NaN,GB
3,112,0.0,35000.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,...,9132d515-dc0e-4494-85ae-20f06eed14f9,112,United States,United States,pop;r&b,112,100.0,NaN,NaN,US
4,2 CHAINZ,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,0.0,0.0,...,dff0d392-4cd5-4052-9fbb-f485df3891e5,2 CHAINZ,United States,United States,hip-hop,2 CHAINZ,100.0,NaN,NaN,US


In [28]:
df_wide.to_csv('data/album_sales_wide_3.csv', index=False)

### Cleanup genre

In [29]:
df_wide = pd.read_csv('data/album_sales_wide_3.csv', index_col='Artist')
df_wide.shape

(2931, 122)

In [30]:
df_wide.head()

,Sales|Argentina,Sales|Australia,Sales|Austria,Sales|Belgium,Sales|Brazil,Sales|Bulgaria,Sales|Canada,Sales|Chile,Sales|China,Sales|Colombia,...,ID,Artist_name,Country,Area,Genre,Artist OG,artist_match,city,iso2,Country Code
Artist,,,,,,,,,,,,,,,,,,,,,
10 YEARS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,b18bc9c4-6f22-4f1b-a918-e9c86a39fe7a,10 YEARS,United States,United States,metal;rock;alternative,10 YEARS,100.0,NaN,NaN,US
10 000 MANIACS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,b9a06530-1241-4162-836f-7b8e79deaa58,10 000 MANIACS,United States,United States,indie;rock;alternative;pop,"10,000 MANIACS",100.0,NaN,NaN,US
10CC,0.0,0.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,...,f37c537b-3557-4031-bfd6-ab63ced32854,10CC,United Kingdom,United Kingdom,rock;pop,10CC,100.0,NaN,NaN,GB
112,0.0,35000.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,...,9132d515-dc0e-4494-85ae-20f06eed14f9,112,United States,United States,pop;r&b,112,100.0,NaN,NaN,US
2 CHAINZ,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,0.0,0.0,0.0,...,dff0d392-4cd5-4052-9fbb-f485df3891e5,2 CHAINZ,United States,United States,hip-hop,2 CHAINZ,100.0,NaN,NaN,US


In [31]:
master_genres = {
    'indie':'rock',
    'alternative':'rock',
    'rap':'hip-hop',
    'hip hop':'hip-hop',
    'new-wave':'pop'
}

In [32]:
df_wide['Genre'] = df_wide['Genre'].replace(master_genres, regex=True)
df_wide = df_wide[~df_wide['Genre'].isna()]
df_wide['Genre'] = df_wide['Genre'].str.split(";").map(set).str.join(", ")

In [33]:
#df_wide.loc['BABY BASH']['Genre']

In [34]:
df_wide = df_wide[~df_wide['Genre'].isna()]

In [35]:
df_wide.head()

,Sales|Argentina,Sales|Australia,Sales|Austria,Sales|Belgium,Sales|Brazil,Sales|Bulgaria,Sales|Canada,Sales|Chile,Sales|China,Sales|Colombia,...,ID,Artist_name,Country,Area,Genre,Artist OG,artist_match,city,iso2,Country Code
Artist,,,,,,,,,,,,,,,,,,,,,
10 YEARS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,b18bc9c4-6f22-4f1b-a918-e9c86a39fe7a,10 YEARS,United States,United States,"rock, metal",10 YEARS,100.0,NaN,NaN,US
10 000 MANIACS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,b9a06530-1241-4162-836f-7b8e79deaa58,10 000 MANIACS,United States,United States,"rock, pop","10,000 MANIACS",100.0,NaN,NaN,US
10CC,0.0,0.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,...,f37c537b-3557-4031-bfd6-ab63ced32854,10CC,United Kingdom,United Kingdom,"rock, pop",10CC,100.0,NaN,NaN,GB
112,0.0,35000.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,...,9132d515-dc0e-4494-85ae-20f06eed14f9,112,United States,United States,"r&b, pop",112,100.0,NaN,NaN,US
2 CHAINZ,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,0.0,0.0,0.0,...,dff0d392-4cd5-4052-9fbb-f485df3891e5,2 CHAINZ,United States,United States,hip-hop,2 CHAINZ,100.0,NaN,NaN,US


In [36]:
df_wide.to_csv('data/album_sales_wide_4.csv')

## Feature engineering

In [37]:
df_wide = pd.read_csv('data/album_sales_wide_4.csv', index_col='Artist')

In [38]:
def domestic_sales(artist, scaled=False):
    my_line = df_wide.loc[artist]
    my_country = my_line['Country']
    
    if scaled:
        col = 'Scaled|' + my_country
    else:
        col = 'Sales|' + my_country
        
    try:
        my_domestic = my_line[col]
    except:
        my_domestic = 0 #np.nan
  
    return my_domestic

In [39]:
df_wide['Domestic'] = [domestic_sales(a) for a in df_wide.index]
df_wide['Domestic Scaled'] = [domestic_sales(a, scaled=True) for a in df_wide.index]
df_wide['Intl'] = df_wide['Total'] - df_wide['Domestic']
df_wide['Intl Scaled'] = df_wide['Total Scaled'] - df_wide['Domestic Scaled']
df_wide['% Domestic'] = round(df_wide['Domestic']/df_wide['Total'], 3)
df_wide['% Domestic Scaled'] = round(df_wide['Domestic Scaled']/df_wide['Total Scaled'], 3)

In [40]:
df_wide[['Domestic', 'Domestic Scaled','Total','Total Scaled','% Domestic', '% Domestic Scaled']].describe()

,Domestic,Domestic Scaled,Total,Total Scaled,% Domestic,% Domestic Scaled
count,2.465000e+03,2.465000e+03,2.465000e+03,2.465000e+03,2465.000000,2465.000000
mean,4.038843e+06,4.258945e+05,6.617036e+06,3.495316e+06,0.726910,0.497897
std,8.791486e+06,3.938233e+06,1.431518e+07,1.729920e+07,0.344162,0.384262
min,0.000000e+00,0.000000e+00,5.000000e+05,1.000000e+04,0.000000,0.000000
25%,5.000000e+05,3.000000e+04,1.000000e+06,5.000000e+04,0.542000,0.121000
50%,1.435000e+06,7.000000e+04,2.060000e+06,2.300000e+05,0.907000,0.435000
75%,3.500000e+06,2.200000e+05,5.685000e+06,1.130000e+06,1.000000,1.000000
max,1.394500e+08,1.543700e+08,2.142014e+08,3.836000e+08,1.000000,1.000000


In [41]:
df_wide.head()

,Sales|Argentina,Sales|Australia,Sales|Austria,Sales|Belgium,Sales|Brazil,Sales|Bulgaria,Sales|Canada,Sales|Chile,Sales|China,Sales|Colombia,...,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,Intl,Intl Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
10 YEARS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,NaN,NaN,US,500000.0,10000.0,0.0,0.0,1.000,1.000
10 000 MANIACS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,100.0,NaN,NaN,US,8000000.0,220000.0,180000.0,30000.0,0.978,0.880
10CC,0.0,0.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,...,100.0,NaN,NaN,GB,1560000.0,250000.0,405300.0,610000.0,0.794,0.291
112,0.0,35000.0,0.0,0.0,0.0,0.0,150000.0,0.0,0.0,0.0,...,100.0,NaN,NaN,US,5500000.0,150000.0,245000.0,100000.0,0.957,0.600
2 CHAINZ,0.0,0.0,0.0,0.0,0.0,0.0,40000.0,0.0,0.0,0.0,...,100.0,NaN,NaN,US,2000000.0,60000.0,40000.0,20000.0,0.980,0.750


In [42]:
artist = 'THE WHO'
line = df_wide.loc[df_wide.index == artist]
line.loc[:, (line != 0).all(axis=0)]

,Sales|Canada,Sales|France,Sales|Italy,Sales|Japan,Sales|New Zealand,Sales|United Kingdom,Sales|United States,Scaled|Canada,Scaled|France,Scaled|Italy,...,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,Intl,Intl Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
THE WHO,350000.0,400000.0,50000.0,49370.0,7500.0,3240000.0,21825000.0,140000.0,160000.0,200000.0,...,100.0,NaN,NaN,GB,3240000.0,510000.0,22681870.0,1170000.0,0.125,0.304


In [43]:
artist = 'SIGUR ROS'
line = df_wide.loc[df_wide.index == artist]
line.loc[:, (line != 0).all(axis=0)]

,Sales|Belgium,Sales|Denmark,Sales|Iceland,Sales|Portugal,Sales|South Korea,Sales|United Kingdom,Scaled|Belgium,Scaled|Denmark,Scaled|Iceland,Scaled|Portugal,...,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,Intl,Intl Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
SIGUR ROS,65000.0,20000.0,20463.0,10000.0,3663.0,720000.0,380000.0,100000.0,5520000.0,110000.0,...,100.0,NaN,NaN,IS,20463.0,5520000.0,818663.0,700000.0,0.024,0.887


In [44]:
artist = 'RED HOT CHILI PEPPERS'
line = df_wide.loc[df_wide.index == artist]
line.loc[:, (line != 0).all(axis=0)]

,Sales|Argentina,Sales|Australia,Sales|Austria,Sales|Belgium,Sales|Brazil,Sales|Canada,Sales|Chile,Sales|Croatia,Sales|Czech Republic,Sales|Denmark,...,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,Intl,Intl Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
RED HOT CHILI PEPPERS,390000.0,2380000.0,267500.0,215000.0,900000.0,2104000.0,45000.0,10000.0,25000.0,290000.0,...,100.0,NaN,NaN,US,25623000.0,720000.0,25802142.0,92420000.0,0.498,0.008


In [45]:
artist = 'OASIS'
line = df_wide.loc[df_wide.index == artist]
line.loc[:, (line != 0).all(axis=0)]
#line.loc[:, (line == line['Max Intl Scaled'][0]).all(axis=0)]

,Sales|Argentina,Sales|Australia,Sales|Austria,Sales|Belgium,Sales|Canada,Sales|Denmark,Sales|Finland,Sales|France,Sales|Germany,Sales|Hong Kong,...,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,Intl,Intl Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
OASIS,130000.0,770000.0,25000.0,50000.0,1300000.0,120000.0,53639.0,700000.0,500000.0,30000.0,...,100.0,NaN,NaN,GB,16100000.0,2540000.0,13897317.0,17040000.0,0.537,0.13


In [46]:
artist = 'BEAUTIFUL SOUTH'
line = df_wide.loc[df_wide.index == artist]
line.loc[:, (line != 0).all(axis=0)]

,Sales|United Kingdom,Scaled|United Kingdom,Total,Total Scaled,ID,Artist_name,Country,Area,Genre,Artist OG,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,
BEAUTIFUL SOUTH,6608000.0,1040000.0,6608000.0,1040000.0,eec43ebc-815a-4105-b1cb-205b8f57b4fa,THE BEAUTIFUL SOUTH,United Kingdom,United Kingdom,"rock, pop",BEAUTIFUL SOUTH,88.0,NaN,NaN,GB,6608000.0,1040000.0,1.0,1.0


In [47]:
artist = 'BUSH'
line = df_wide.loc[df_wide.index == artist]
line.loc[:, (line != 0).all(axis=0)]

,Sales|Australia,Sales|Canada,Sales|New Zealand,Sales|United Kingdom,Sales|United States,Scaled|Australia,Scaled|Canada,Scaled|New Zealand,Scaled|United Kingdom,Scaled|United States,...,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,Intl,Intl Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
BUSH,175000.0,1300000.0,37500.0,160000.0,10543000.0,160000.0,540000.0,240000.0,30000.0,290000.0,...,100.0,NaN,NaN,GB,160000.0,30000.0,12055500.0,1230000.0,0.013,0.024


In [48]:
artist = 'LYNYRD SKYNYRD'
line = df_wide.loc[df_wide.index == artist]
line.loc[:, (line != 0).all(axis=0)]

,Sales|Australia,Sales|Canada,Sales|New Zealand,Sales|United Kingdom,Sales|United States,Scaled|Australia,Scaled|Canada,Scaled|New Zealand,Scaled|United Kingdom,Scaled|United States,...,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,Intl,Intl Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
LYNYRD SKYNYRD,35000.0,366000.0,7500.0,540000.0,27002667.0,30000.0,150000.0,50000.0,90000.0,750000.0,...,100.0,NaN,NaN,US,27002667.0,750000.0,948500.0,320000.0,0.966,0.701


In [49]:
artist = 'BLUR'
line = df_wide.loc[df_wide.index == artist]
line.loc[:, (line != 0).all(axis=0)]
#line.loc[:, (line == line['Max Intl Scaled'][0]).all(axis=0)]

,Sales|Australia,Sales|Canada,Sales|Denmark,Sales|France,Sales|Japan,Sales|Latvia,Sales|New Zealand,Sales|Norway,Sales|Spain,Sales|Sweden,...,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,Intl,Intl Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
BLUR,70000.0,250000.0,20000.0,100000.0,666989.0,4000.0,37500.0,25000.0,250000.0,50000.0,...,100.0,NaN,NaN,GB,4400000.0,690000.0,1973489.0,2970000.0,0.69,0.189


In [50]:
artist = 'BOWLING FOR SOUP'
line = df_wide.loc[df_wide.index == artist]
line.loc[:, (line != 0).all(axis=0)]

,Sales|United Kingdom,Sales|United States,Scaled|United Kingdom,Scaled|United States,Total,Total Scaled,ID,Artist_name,Country,Area,...,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,Intl,Intl Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
BOWLING FOR SOUP,120000.0,500000.0,20000.0,10000.0,620000.0,30000.0,bcb95908-5c6e-453a-97fc-d69003d250ff,BOWLING FOR SOUP,United States,United States,...,100.0,NaN,NaN,US,500000.0,10000.0,120000.0,20000.0,0.806,0.333


In [51]:
artist = 'THE KILLERS'
line = df_wide.loc[df_wide.index == artist]
line.loc[:, (line != 0).all(axis=0)]
#line.loc[:, (line == line['Max Intl Scaled'][0]).all(axis=0)]

,Sales|Argentina,Sales|Australia,Sales|Austria,Sales|Belgium,Sales|Canada,Sales|Denmark,Sales|France,Sales|Germany,Sales|Greece,Sales|Ireland,...,artist_match,city,iso2,Country Code,Domestic,Domestic Scaled,Intl,Intl Scaled,% Domestic,% Domestic Scaled
Artist,,,,,,,,,,,,,,,,,,,,,
THE KILLERS,40000.0,595000.0,10000.0,50000.0,580000.0,30000.0,100000.0,400000.0,3000.0,150000.0,...,100.0,NaN,NaN,US,4500000.0,130000.0,8576683.0,6070000.0,0.344,0.021


In [52]:
df_wide.to_csv('data/album_sales_wide_5.csv')